# Notebook for first experiments with Yahoo Finance package for python

Importing libraries

In [51]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

from scipy.stats import gmean

One ticker symbol

In [14]:
dat = yf.Ticker("PCO.WA")
dat.get_info()

{'address1': 'Capital House',
 'address2': '14th Floor 25 Chapel Street',
 'city': 'London',
 'zip': 'NW1 5DH',
 'country': 'United Kingdom',
 'phone': '44 20 3735 9210',
 'website': 'https://www.pepcogroup.eu',
 'industry': 'Discount Stores',
 'industryKey': 'discount-stores',
 'industryDisp': 'Discount Stores',
 'sector': 'Consumer Defensive',
 'sectorKey': 'consumer-defensive',
 'sectorDisp': 'Consumer Defensive',
 'longBusinessSummary': "Pepco Group N.V. operates as a discount variety retailer in the United Kingdom, the Republic of Ireland, Poland, and rest of Europe. The company provides clothing's, such as kids and babywear; homeware-led general merchandise; and fast-moving consumer goods. It also offers home décors, chilled and frozen foods, toys, sweets, snacks, beverages, household cleaning products, cosmetics, office supplies, pet products, gardening, health, beauty, and seasonal products. In addition, the company provides direct product sourcing, product development, and tec

In [40]:
relevant_keys = [
    'city', 'country', 'industryKey', 'sectorKey', 'fullTimeEmployees', 'currency', 'tradeable', 'quoteType',
    'financialCurrency', 'region', 'fullExchangeName', 'exchange', 'exchangeTimezoneName', 'market', 'marketCap',
    'shortName', 'ebitda', 'totalDebt', 'debtToEquity', 'totalRevenue'] # market cap

In [53]:
def imput_asset_info(relevant_info, df):
    """
    Enriches the input DataFrame with additional asset information from Yahoo Finance.

    Parameters:
    - relevant_info: List of keys to fetch from yfinance .info dictionary
    - df: DataFrame that must include a 'Ticker' column

    Returns:
    - Merged DataFrame with additional asset information
    """
    columns = ['Ticker'] + relevant_info
    distinct_tickers = df['Ticker'].unique()
    rows = []

    for ticker in distinct_tickers:
        try:
            info_dict = yf.Ticker(ticker).info
            data_row = [ticker] + [info_dict.get(field, None) for field in relevant_info]
            rows.append(data_row)
        except Exception as e:
            print(f"Warning: Could not retrieve info for {ticker}: {e}")
            continue

    df_additional_info = pd.DataFrame(rows, columns=columns)
    return pd.merge(df, df_additional_info, how='left', on='Ticker')

In [ ]:
data = {
    "Ticker": ["AAPL", "GOOGL", "MSFT", "TSLA", "AMZN"],
    "Company": ["Apple Inc.", "Alphabet Inc.", "Microsoft Corp.", "Tesla Inc.", "Amazon.com Inc."],
    "Price": [165.3, 142.7, 310.1, 182.4, 127.8],
    "Sector": ["Technology", "Technology", "Technology", "Automotive", "Consumer Discretionary"]
}

df = pd.DataFrame(data)

In [54]:
imput_asset_info(relevant_info=relevant_keys, df=df)

Ticker          Company  Price                  Sector           city  \
0   AAPL       Apple Inc.  165.3              Technology      Cupertino   
1  GOOGL    Alphabet Inc.  142.7              Technology  Mountain View   
2   MSFT  Microsoft Corp.  310.1              Technology        Redmond   
3   TSLA       Tesla Inc.  182.4              Automotive         Austin   
4   AMZN  Amazon.com Inc.  127.8  Consumer Discretionary        Seattle   

         country                   industryKey               sectorKey  \
0  United States          consumer-electronics              technology   
1  United States  internet-content-information  communication-services   
2  United States       software-infrastructure              technology   
3  United States            auto-manufacturers       consumer-cyclical   
4  United States               internet-retail       consumer-cyclical   

   fullTimeEmployees currency  ...  fullExchangeName exchange  \
0             150000      USD  ...          NasdaqGS      NMS   
1             183323      USD  ...          NasdaqGS      NMS   
2             228000      USD  ...          NasdaqGS      NMS   
3             125665      USD  ...          NasdaqGS      NMS   
4            1556000      USD  ...          NasdaqGS      NMS   

  exchangeTimezoneName     market      marketCap              shortName  \
0     America/New_York  us_market  3073521745920             Apple Inc.   
1     America/New_York  us_market  1907418464256          Alphabet Inc.   
2     America/New_York  us_market  2783207751680  Microsoft Corporation   
3     America/New_York  us_market   806510198784            Tesla, Inc.   
4     America/New_York  us_market  1916599533568       Amazon.com, Inc.   

         ebitda     totalDebt  debtToEquity  totalRevenue  
0  137352003584   96798998528       145.000  395760009216  
1  129496997888   28137000960         8.655  350018011136  
2  142041006080  102909001728        33.998  261802000384  
3   12549999616    7528999936         9.983   95724003328  
4  120467996672  155400994816        54.342  637959012352  

[5 rows x 24 columns]

In [78]:
values = [dat.info[k] for k in relevant_keys]

In [79]:
values

['London',
 'United Kingdom',
 'discount-stores',
 'consumer-defensive',
 29462,
 'PLN',
 False,
 'EQUITY',
 'EUR',
 'US',
 'Warsaw',
 'WSE',
 'Europe/Warsaw',
 'pl_market',
 'PEPCO']

In [3]:
wig20 = [
    "ALR.WA", "ALE.WA", "BDX.WA", "CCC.WA", "CDR.WA",
    "CPS.WA", "DNP.WA", "KTY.WA", "JSW.WA", "KGH.WA",
    "KRU.WA", "LPP.WA", "MBK.WA", "OPL.WA", "PEO.WA",
    "PGE.WA", "PKN.WA", "PKO.WA", "PZU.WA", "PCO.WA"
]

This function gives us all information stored in a security, is a dict type

In [82]:
data = yf.Tickers(wig20)

In [ ]:
data.tickers[ticker].fast_info['currency']

KeyError: 1

In [4]:
start = datetime.datetime(2024,11,15)
end = datetime.datetime(2025,4,4)

In [5]:
def get_stocks(tickers, start, end):
    df = yf.download(tickers, start, end)[["Close"]].reset_index()
    df.columns = [col[1] if col[0] == 'Close' else 'Date' for col in df.columns]
    return df

In [75]:
data = yf.download(wig20, period='6y', group_by='column')
    
        
data = data.reset_index()
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

[*********************100%***********************]  20 of 20 completed


In [76]:
data.head()


Price       Close                                                            \
Ticker     ALE.WA     ALR.WA      BDX.WA      CCC.WA      CDR.WA     CPS.WA   
Date                                                                          
2019-04-24    NaN  57.345783  101.512238  222.230484  199.159058  22.301405   
2019-04-25    NaN  56.437050  102.329788  229.405640  197.933182  22.065054   
2019-04-26    NaN  56.389225  102.193520  230.202881  199.536270  22.554640   
2019-04-29    NaN  56.484879  100.422165  205.687759  200.384949  22.368935   
2019-04-30    NaN  56.102257  100.422165  206.684311  202.648117  22.368935   

Price                                                      ... Volume         \
Ticker          DNP.WA     JSW.WA      KGH.WA      KRU.WA  ... KTY.WA LPP.WA   
Date                                                       ...                 
2019-04-24  126.400002  56.448448  100.322960  133.771790  ...    620   3621   
2019-04-25  124.900002  56.874294   97.983345  132.362793  ...   3520   1731   
2019-04-26  127.000000  56.164547   98.872398  141.645599  ...    503   1287   
2019-04-29  126.000000  55.738701   98.685234  139.822189  ...   1365    993   
2019-04-30  127.000000  54.934322   96.252014  143.800522  ...    296   1649   

Price                                                                          
Ticker     MBK.WA   OPL.WA PCO.WA   PEO.WA   PGE.WA  PKN.WA   PKO.WA   PZU.WA  
Date                                                                           
2019-04-24  24977  1565428    NaN  1346956  2426609  707948  2598172  1072220  
2019-04-25  12871   963644    NaN   507214  1815669  809071  1507611   889308  
2019-04-26  19670   994793    NaN   427001  1389663  577813  1087240   645464  
2019-04-29  19713   806089    NaN   226199  1592924  435147  1348469   847291  
2019-04-30  14952  1964384    NaN   777053  1703505  817421  3238208  1412784  

[5 rows x 100 columns]

 średnie do 5 lat, począwszy od dnia: dzień, tydzień, miesiąc, kwartał, pół-rocze, rok, 2, 3, 4, 5

In [29]:
data.sort_values('Date', ascending=False)['ALE.WA'].iloc[:7]

Date
2025-04-24    33.419998
2025-04-23    33.099998
2025-04-22    31.570000
2025-04-17    31.100000
2025-04-16    30.879999
2025-04-15    31.510000
2025-04-14    31.165001
Name: ALE.WA, dtype: float64

In [ ]:
def calculate_averages(df, columns, type='arithmetic'):
    # we can adjust a function so we can aggregate any values in any column i.e. Close price or Volume
    df = df[column]
    # we assume our df is sorted by descending order of dates (latest is at the top), if not we can repair it
    df = df.sort_values('Date', ascending=False)
    # gather all the tickers
    tickers = df.columns.to_list()
    # store all required periods (as number of days)
    periods = [1, 5, 63, 90, 126, 252, 504, 756, 1008, 1260] # it can be adjusted to what we consider a 'week' in a trading year
    
    # handling column names
    if column == 'Volume':
        base = f"average{column}Last{{}}"
    base = f"average{column}PriceLast{{}}"

    columns = [
        'Ticker',
        base.format('Day'),
        base.format('Week'),
        base.format('Month'),
        base.format('Quarter'),
        base.format('HalfYear'),
        base.format('1Y'),
        base.format('2Y'),
        base.format('3Y'),
        base.format('4Y'),
        base.format('5Y')
    ]

    data_rows = []
    for ticker in tickers:
        row = [ticker]
        for period in periods:
            sliced = df[ticker].iloc[:period]  # latest `period` rows
            if type == 'arithmetic':
                row.append(sliced.mean(skipna=True)) # make sure we skip NaN
            elif type == 'geometric':
                row.append(gmean(sliced.dropna()))
            else:
                raise ValueError('Invalid type of mean!')
        data_rows.append(row)

    # Create final DataFrame
    aggregated_df = pd.DataFrame(data_rows, columns=columns)

    return aggregated_df


In [71]:
calculate_averages(data, 'Close')

Ticker  AverageClosePriceLastDay  AverageClosePriceLastWeek  \
0   ALE.WA                 33.419998                  32.013999   
1   ALR.WA                111.050003                 108.710001   
2   BDX.WA                620.200012                 627.600000   
3   CCC.WA                226.399994                 229.439999   
4   CDR.WA                224.699997                 223.600000   
5   CPS.WA                 17.780001                  17.766000   
6   DNP.WA                519.200012                 511.940002   
7   JSW.WA                 24.110001                  23.692000   
8   KGH.WA                124.449997                 118.639999   
9   KRU.WA                388.600006                 381.159998   
10  KTY.WA                852.000000                 823.800000   
11  LPP.WA              15315.000000               15383.655469   
12  MBK.WA                868.400024                 825.360010   
13  OPL.WA                  9.640000                   9.597600   
14  PCO.WA                 17.275000                  16.551000   
15  PEO.WA                191.600006                 183.050000   
16  PGE.WA                  7.942000                   7.600400   
17  PKN.WA                 67.870003                  65.700001   
18  PKO.WA                 76.519997                  73.059999   
19  PZU.WA                 58.860001                  57.696001   

    AverageClosePriceLastMonth  AverageClosePriceLastQuarter  \
0                    30.206428                     29.431611   
1                   102.131746                     98.119778   
2                   563.860317                    534.728888   
3                   201.380952                    196.628888   
4                   217.438095                    211.040556   
5                    15.294683                     15.035111   
6                   471.766667                    453.566666   
7                    24.388730                     23.779111   
8                   128.369841                    126.511111   
9                   405.919048                    411.534445   
10                  808.142857                    775.450000   
11                16847.566003                  16448.209234   
12                  744.295241                    689.080002   
13                    8.580825                      8.224378   
14                   16.356417                     16.322851   
15                  171.465874                    163.292778   
16                    7.304413                      6.938067   
17                   62.460000                     58.652667   
18                   71.025079                     67.964889   
19                   54.247778                     52.190778   

    AverageClosePriceLastHalfYear  AverageClosePriceLast1Y  \
0                       30.181627                33.371468   
1                       96.001905                96.148791   
2                      527.355554               587.669839   
3                      195.415079               164.746627   
4                      197.906746               175.182715   
5                       14.484762                13.521401   
6                      429.614285               398.311508   
7                       24.451429                26.178492   
8                      130.474206               137.084180   
9                      415.811905               429.741733   
10                     759.321429               775.280423   
11                   16032.319809             15667.237293   
12                     653.542859               640.246033   
13                       8.108651                 8.080413   
14                      16.437676                18.191410   
15                     157.886112               158.252537   
16                       6.888540                 6.925611   
17                      56.849762                58.412856   
18                      64.816508                60.580778   
19         

In [64]:
data = yf.download(wig20, group_by='Ticker', period= '1d')
data.head()

[*********************100%***********************]  20 of 20 completed


Ticker       LPP.WA                                       PZU.WA             \
Price          Open     High      Low    Close Volume       Open       High   
Date                                                                          
2025-04-23  15615.0  15845.0  15615.0  15760.0    501  58.119999  58.799999   

Ticker                                    ... CCC.WA                          \
Price             Low      Close  Volume  ...   Open        High         Low   
Date                                      ...                                  
2025-04-23  57.939999  58.639999  205305  ...  228.5  231.399994  226.699997   

Ticker                   MBK.WA                                   
Price       Close Volume   Open   High         Low  Close Volume  
Date                                                              
2025-04-23  230.0  31403  831.0  839.0  830.799988  832.0   2856  

[1 rows x 100 columns]

In [ ]:

data = data.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)

data['Change'] = data['Close'] - data['Open']
data['Daily Return'] = (data['Close'] - data['Open']) / data['Open']
data['Range'] = data['High'] - data['Low']
data['Volatility Ratio'] = data['Range'] / data['Open']
data['Upper Wick'] = data['High'] - np.maximum(data['Open'], data['Close'])
data['Lower Wick'] = np.minimum(data['Open'], data['Close']) - data['Low']
data['Candle type'] = np.where(data['Close'] > data['Open'], 1, 0)  # 1 means bullish, 0 means bearish

data.head(20)

This function gives us important dates of a security

In [24]:
dat.calendar 

{'Dividend Date': datetime.date(2025, 6, 12),
 'Ex-Dividend Date': datetime.date(2025, 5, 15),
 'Earnings Date': [datetime.date(2025, 4, 23), datetime.date(2025, 4, 28)],
 'Earnings High': 3.56,
 'Earnings Low': 3.16,
 'Earnings Average': 3.22562,
 'Revenue High': 69897682310,
 'Revenue Low': 67140000000,
 'Revenue Average': 68536220780}

In [25]:
dat.analyst_price_targets

{'current': 359.84,
 'high': 650.0,
 'low': 415.0,
 'mean': 503.86472,
 'median': 500.0}

In [26]:
dat.quarterly_income_stmt

2024-12-31  \
Tax Effect Of Unusual Items                          -203220000.0   
Tax Rate For Calcs                                           0.18   
Normalized EBITDA                                   37915000000.0   
Total Unusual Items                                 -1129000000.0   
Total Unusual Items Excluding Goodwill              -1129000000.0   
Net Income From Continuing Operation Net Minori...  24108000000.0   
Reconciled Depreciation                              6827000000.0   
Reconciled Cost Of Revenue                          21799000000.0   
EBITDA                                              36786000000.0   
EBIT                                                29959000000.0   
Net Interest Income                                     6000000.0   
Interest Expense                                      594000000.0   
Interest Income                                       600000000.0   
Normalized Income                                   25033780000.0   
Net Income From Continuing And Discontinued Ope...  24108000000.0   
Total Expenses                                      37979000000.0   
Total Operating Income As Reported                  31653000000.0   
Diluted Average Shares                               7468000000.0   
Basic Average Shares                                 7435000000.0   
Diluted EPS                                                  3.23   
Basic EPS                                                    3.24   
Diluted NI Availto Com Stockholders                 24108000000.0   
Net Income Common Stockholders                      24108000000.0   
Net Income                                          24108000000.0   
Net Income Including Noncontrolling Interests       24108000000.0   
Net Income Continuous Operations                    24108000000.0   
Tax Provision                                        5257000000.0   
Pretax Income                                       29365000000.0   
Other Income Expense                                -2294000000.0   
Other Non Operating Income Expenses                 -1165000000.0   
Special Income Charges                               -867000000.0   
Write Off                                             867000000.0   
Gain On Sale Of Security                             -262000000.0   
Net Non Operating Interest Income Expense               6000000.0   
Interest Expense Non Operating                        594000000.0   
Interest Income Non Operating                         600000000.0   
Operating Income                                    31653000000.0   
Operating Expense                                   16180000000.0   
Research And Development                             7917000000.0   
Selling General And Administration                   8263000000.0   
Selling And Marketing Expense                        6440000000.0   
General And Administrative Expense                   1823000000.0   
Other Gand A                                         1823000000.0   
Gross Profit                                        47833000000.0   
Cost Of Revenue                                     21799000000.0   
Total Revenue                                       69632000000.0   
Operating Revenue                                   69632000000.0   

                                                       2024-09-30  \
Tax Effect Of Unusual Items                            57190000.0   
Tax Rate For Calcs                                           0.19   
Normalized EBITDA                                   37933000000.0   
Total Unusual Items                                   301000000.0   
Total Unusual Items Excluding Goodwill                301000000.0   
Net Income From Continuing Operation Net Minori...  24667000000.0   
Reconciled Depreciation                              7383000000.0   
Reconciled Cost Of Revenue                          20099000000.0   
EBITDA                                              38234000000.0   
EBIT                                                

Period must be of the format 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max, etc.
This function gives us information about historical values of a security

In [27]:
dat.history(period='5d')

Open        High         Low       Close  \
Date                                                                        
2025-03-31 00:00:00-04:00  372.540009  377.070007  367.239990  375.390015   
2025-04-01 00:00:00-04:00  374.649994  382.850006  373.230011  382.190002   
2025-04-02 00:00:00-04:00  377.970001  385.079987  376.619995  382.140015   
2025-04-03 00:00:00-04:00  374.790009  377.480011  369.350006  373.109985   
2025-04-04 00:00:00-04:00  364.130005  374.589996  359.480011  359.839996   

                             Volume  Dividends  Stock Splits  
Date                                                          
2025-03-31 00:00:00-04:00  35184700        0.0           0.0  
2025-04-01 00:00:00-04:00  19689500        0.0           0.0  
2025-04-02 00:00:00-04:00  16092600        0.0           0.0  
2025-04-03 00:00:00-04:00  30198000        0.0           0.0  
2025-04-04 00:00:00-04:00  49138700        0.0           0.0

In [28]:
dat.option_chain(dat.options[0]).calls

contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0   MSFT250411C00230000 2025-04-04 14:53:22+00:00   230.0     137.15  128.15   
1   MSFT250411C00260000 2025-03-07 20:54:19+00:00   260.0     133.50   98.15   
2   MSFT250411C00290000 2025-03-28 14:04:20+00:00   290.0      97.70   68.70   
3   MSFT250411C00300000 2025-04-04 17:46:49+00:00   300.0      66.54   58.85   
4   MSFT250411C00305000 2025-03-18 15:48:24+00:00   305.0      78.48   58.70   
5   MSFT250411C00310000 2025-04-04 19:07:18+00:00   310.0      52.00   53.85   
6   MSFT250411C00320000 2025-04-04 18:31:40+00:00   320.0      44.05   44.50   
7   MSFT250411C00325000 2025-04-04 19:32:17+00:00   325.0      41.27   40.30   
8   MSFT250411C00330000 2025-04-04 17:38:26+00:00   330.0      34.10   36.20   
9   MSFT250411C00335000 2025-04-04 15:37:12+00:00   335.0      34.90   31.95   
10  MSFT250411C00340000 2025-04-04 19:33:47+00:00   340.0      24.35   26.80   
11  MSFT250411C00345000 2025-04-04 19:12:31+00:00   345.0      20.40   23.80   
12  MSFT250411C00350000 2025-04-04 19:41:05+00:00   350.0      20.00   19.85   
13  MSFT250411C00355000 2025-04-04 19:40:41+00:00   355.0      13.40   16.15   
14  MSFT250411C00360000 2025-04-04 19:59:57+00:00   360.0      10.35   12.80   
15  MSFT250411C00362500 2025-04-04 19:58:34+00:00   362.5       8.95   11.25   
16  MSFT250411C00365000 2025-04-04 19:59:57+00:00   365.0       7.70    9.80   
17  MSFT250411C00367500 2025-04-04 19:59:33+00:00   367.5       6.45    8.50   
18  MSFT250411C00370000 2025-04-04 19:59:19+00:00   370.0       5.34    7.30   
19  MSFT250411C00372500 2025-04-04 19:59:08+00:00   372.5       4.43    6.20   
20  MSFT250411C00375000 2025-04-04 19:59:15+00:00   375.0       3.75    5.25   
21  MSFT250411C00377500 2025-04-04 19:57:06+00:00   377.5       3.10    4.35   
22  MSFT250411C00380000 2025-04-04 19:59:34+00:00   380.0       2.42    3.60   
23  MSFT250411C00382500 2025-04-04 19:57:44+00:00   382.5       1.95    3.00   
24  MSFT250411C00385000 2025-04-04 19:59:43+00:00   385.0       1.43    2.44   
25  MSFT250411C00387500 2025-04-04 19:52:58+00:00   387.5       1.38    1.98   
26  MSFT250411C00390000 2025-04-04 19:59:50+00:00   390.0       0.80    1.60   
27  MSFT250411C00392500 2025-04-04 19:52:41+00:00   392.5       0.81    1.28   
28  MSFT250411C00395000 2025-04-04 19:59:43+00:00   395.0       0.45    1.02   
29  MSFT250411C00397500 2025-04-04 19:59:02+00:00   397.5       0.41    0.82   
30  MSFT250411C00400000 2025-04-04 19:58:28+00:00   400.0       0.27    0.65   
31  MSFT250411C00402500 2025-04-04 19:51:56+00:00   402.5       0.21    0.51   
32  MSFT250411C00405000 2025-04-04 19:53:36+00:00   405.0       0.17    0.40   
33  MSFT250411C00407500 2025-04-04 19:13:15+00:00   407.5       0.13    0.32   
34  MSFT250411C00410000 2025-04-04 19:41:44+00:00   410.0       0.17    0.25   
35  MSFT250411C00415000 2025-04-04 18:49:25+00:00   415.0       0.07    0.16   
36  MSFT250411C00420000 2025-04-04 19:53:07+00:00   420.0       0.04    0.09   
37  MSFT250411C00425000 2025-04-04 19:53:36+00:00   425.0       0.08    0.06   
38  MSFT250411C00430000 2025-04-04 19:44:11+00:00   430.0       0.04    0.03   
39  MSFT250411C00435000 2025-04-04 16:21:20+00:00   435.0       0.01    0.01   
40  MSFT250411C00440000 2025-04-04 18:34:50+00:00   440.0       0.01    0.00   
41  MSFT250411C00445000 2025-04-03 19:59:35+00:00   445.0       0.02    0.00   
42  MSFT250411C00450000 2025-04-03 19:18:17+00:00   450.0       0.01    0.00   
43  MSFT250411C00455000 2025-03-26 14:12:00+00:00   455.0       0.01    0.00   
44  MSFT250411C00460000 2025-04-01 17:58:03+00:00   460.0       0.02    0.00   
45  MSFT250411C00465000 2025-03-24 13:45:21+00:00   465.0       0.02    0.00   
46  MSFT250411C00470000 2025-04-02 17:36:58+00:00   470.0       0.01    0.00   
47  MSFT250411C00475000 2025-03-06 16:51:36+00:00   475.0       0.22    0.00   
48  MSFT250411C00480000 2025-03-28 15:50:02+00:00   480.0       0.01    0.00   
49  MSFT2

Multiple ticker symbols

In [29]:
tickers = yf.Tickers('MSFT AAPL GOOG')
tickers.tickers['MSFT'].info
yf.download(['MSFT', 'AAPL', 'GOOG'], period='1mo')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  3 of 3 completed


Price            Close                                High              \
Ticker            AAPL        GOOG        MSFT        AAPL        GOOG   
Date                                                                     
2025-03-05  235.740005  174.790878  401.019989  236.550003  175.550008   
2025-03-06  235.330002  174.011765  396.890015  237.860001  176.528886   
2025-03-07  239.070007  175.550003  393.309998  241.369995  176.698688   
2025-03-10  227.479996  167.809998  380.160004  236.160004  170.449997   
2025-03-11  220.839996  165.979996  380.450012  225.839996  168.654999   
2025-03-12  216.979996  169.000000  383.269989  221.750000  169.529999   
2025-03-13  209.679993  164.729996  378.769989  216.839996  168.119995   
2025-03-14  213.490005  167.619995  388.559998  213.949997  168.250000   
2025-03-17  214.000000  166.570007  388.700012  215.220001  168.460007   
2025-03-18  212.690002  162.669998  383.519989  215.149994  166.440002   
2025-03-19  215.240005  166.279999  387.820007  218.759995  168.132996   
2025-03-20  214.100006  165.050003  386.839996  217.490005  167.029999   
2025-03-21  218.270004  166.250000  391.260010  218.839996  166.470001   
2025-03-24  220.729996  169.929993  393.079987  221.479996  170.500000   
2025-03-25  223.750000  172.789993  395.160004  224.100006  172.910004   
2025-03-26  221.529999  167.139999  389.970001  225.020004  171.940002   
2025-03-27  223.850006  164.080002  390.579987  224.990005  167.440002   
2025-03-28  217.899994  156.059998  378.799988  223.809998  163.809998   
2025-03-31  222.130005  156.229996  375.390015  225.619995  157.130005   
2025-04-01  223.190002  158.880005  382.190002  223.679993  160.080002   
2025-04-02  223.889999  158.860001  382.140015  225.190002  160.274994   
2025-04-03  203.190002  152.630005  373.109985  207.490005  154.686996   
2025-04-04  188.380005  147.740005  359.839996  199.880005  153.089996   

Price                          Low                                Open  \
Ticker            MSFT        AAPL        GOOG        MSFT        AAPL   
Date                                                                     
2025-03-05  401.670013  229.229996  170.735486  388.809998  235.419998   
2025-03-06  402.149994  233.160004  172.311690  392.679993  234.440002   
2025-03-07  394.799988  234.759995  172.053986  385.540009  235.110001   
2025-03-10  386.399994  224.220001  165.565002  377.220001  235.539993   
2025-03-11  386.000000  217.449997  163.240005  376.910004  223.809998   
2025-03-12  385.220001  214.910004  165.479996  378.950012  220.139999   
2025-03-13  385.320007  208.419998  164.070007  377.450012  215.949997   
2025-03-14  390.230011  209.580002  164.509995  379.510010  211.250000   
2025-03-17  392.709991  209.970001  165.809998  385.570007  213.309998   
2025-03-18  387.369995  211.490005  158.800003  381.100006  214.160004   
2025-03-19  389.679993  213.750000  163.050003  384.000000  214.220001   
2025-03-20  391.790009  212.220001  163.139999  383.279999  213.990005   
2025-03-21  391.739990  211.279999  163.029999  382.799988  211.559998   
2025-03-24  395.399994  218.580002  167.440002  389.809998  221.000000   
2025-03-25  396.359985  220.080002  170.550003  392.640015  220.770004   
2025-03-26  395.309998  220.470001  166.860992  388.570007  223.509995   
2025-03-27  392.239990  220.559998  163.850006  387.399994  221.389999   
2025-03-28  389.130005  217.679993  155.339005  376.929993  221.669998   
2025-03-31  377.070007  216.229996  152.210007  367.239990  217.009995   
2025-04-01  382.850006  218.899994  155.255005  373.230011  219.809998   
2025-04-02  385.079987  221.020004  156.529999  376.619995  221.320007   
2025-04-03  377.480011  201.250000  152.179993  369.350006  205.539993   
2025-04-04  374.589996  187.339996  147.539993  359.480011  193.889999   

Price                                  Volume                      
Ticker            GOOG        MSFT       AAPL      GOOG      MSFT  
Date          

In [31]:
spy = yf.Ticker('SPY').funds_data

In [32]:
spy.description

'The trust seeks to achieve its investment objective by holding a portfolio of the common stocks that are included in the index, with the weight of each stock in the portfolio substantially corresponding to the weight of such stock in the index.'

In [33]:
spy.top_holdings

Name  Holding Percent
Symbol                                                 
AAPL                         Apple Inc         0.069954
MSFT                    Microsoft Corp         0.058503
NVDA                       NVIDIA Corp         0.055643
AMZN                    Amazon.com Inc         0.037620
META        Meta Platforms Inc Class A         0.026460
BRK-B   Berkshire Hathaway Inc Class B         0.020465
GOOGL             Alphabet Inc Class A         0.018910
AVGO                      Broadcom Inc         0.016453
GOOG              Alphabet Inc Class C         0.015483
TSLA                         Tesla Inc         0.015204